[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Maria Bolea, 2023 ([m.bolea@se.qmul.ac.uk](mailto:..m.bolea@se.qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This Dashboard is based on UKHSA data.The aim si to compare the hospital admissions with ventilator beds occupancy. 

In [32]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [33]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Loading the initial data

The dashboard retrieves data via a web API, communicating with the UKHSA Coronavirus dashboard using the HTTP protocol and JSON format.

It displays information for England, showcasing number of admissions and ventilator beds in form of charts.

In [36]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("admissions_vs_beds.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

## Wrangle the data

The dashboard processes JSON data from the API, generating visualizations like Table 1.

Table 1: Display of admissions and ventilator beds data.

In [38]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=jsondata['data']

    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()

    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")
        
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
        
    index=pd.date_range(startdate, enddate, freq='D')
    admissions_vs_bedsdf=pd.DataFrame(index=index, columns=['admissions', 'ventilator_beds'])

    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['admissions', 'ventilator_beds']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(admissions_vs_bedsdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                admissions_vs_bedsdf.loc[date, column]=value
                
                # fill in any remaining "holes" due to missing dates
    admissions_vs_bedsdf.fillna(0.0, inplace=True)

    return admissions_vs_bedsdf

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
admissions_vs_bedsdf=wrangle_data(jsondata) # df is the dataframe for plotting
print (admissions_vs_bedsdf)

           admissions ventilator_beds
2020-03-19      586.0             0.0
2020-03-20      691.0             0.0
2020-03-21      777.0             0.0
2020-03-22      859.0             0.0
2020-03-23     1128.0             0.0
...               ...             ...
2023-10-16      450.0             0.0
2023-10-17      544.0             0.0
2023-10-18      544.0             0.0
2023-10-19      507.0             0.0
2023-10-20      416.0             0.0

[1311 rows x 2 columns]


## Download current data

To access the latest data for this dataset, kindly press the button below to update this page.

This action will pull the newest details from the UKHSA Coronavirus database.

In [39]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = ['areaType=nation', 'areaName=England']
    
    struct = {
    "date": "date",
    "admissions": "newAdmissions",
    "ventilator_beds": "covidOccupiedMVBeds"
    }
    api = Cov19API(filters=filters, structure=struct)  # Corrected `filt` to `filters`

    admissions_vs_beds = api.get_json()
    
    return {} # return data read from the API

In [42]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Update', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Download UKHSA data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Update', icon='exclamation-triangle', style=ButtonStyle(), tooltip='D…

## Graphs and Analysis

Graph 1. Comparing hospital admissions and ventilator bed occupancy over time.

In [53]:
series=wdg.SelectMultiple(
    options=['admissions', 'ventilator_beds'],
    value=['admissions', 'ventilator_beds'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'logorithmic'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale])

def admissions_vs_beds_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        admissions_vs_bedsdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(admissions_vs_beds_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)


Output()

Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [UKHSA](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Maria Bolea 2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).